<a href="https://colab.research.google.com/github/NQT2906/CS114.K21/blob/Sarcasm/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **Import library**

In [35]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Read and combine data**

In [36]:
dataset_1 = pd.read_json('https://raw.githubusercontent.com/NQT2906/CS114.K21/Sarcasm/Sarcasm/Sarcasm_Headlines_Dataset.json', lines = True)
dataset_2 = pd.read_json('https://raw.githubusercontent.com/NQT2906/CS114.K21/Sarcasm/Sarcasm/Sarcasm_Headlines_Dataset_v2.json', lines = True)

In [37]:
dataset_1

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [38]:
is_sar = dataset_2.iloc[:, 0]
art = dataset_2.iloc[:, 2]

dataset_2.iloc[:, 0] = art
dataset_2.iloc[:, 2] = is_sar

dataset_2.columns = ['article_link', 'headline', 'is_sarcastic']

In [39]:
dataset_2

,article_link,headline,is_sarcastic
0,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,1
1,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails why congress is falling...,0
2,https://www.huffingtonpost.com/entry/eat-your-...,eat your veggies: 9 deliciously different recipes,0
3,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar from getting t...,1
4,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close to using word 'strea...,1
...,...,...,...
28614,https://www.theonion.com/jews-to-celebrate-ros...,jews to celebrate rosh hashasha or something,1
28615,https://local.theonion.com/internal-affairs-in...,internal affairs investigator disappointed con...,1
28616,https://www.huffingtonpost.com/entry/andrew-ah...,the most beautiful acceptance speech this week...,0
28617,https://www.theonion.com/mars-probe-destroyed-...,mars probe destroyed by orbiting spielberg-gat...,1


In [40]:
dataset = pd.concat([dataset_1, dataset_2])

In [41]:
dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28614,https://www.theonion.com/jews-to-celebrate-ros...,jews to celebrate rosh hashasha or something,1
28615,https://local.theonion.com/internal-affairs-in...,internal affairs investigator disappointed con...,1
28616,https://www.huffingtonpost.com/entry/andrew-ah...,the most beautiful acceptance speech this week...,0
28617,https://www.theonion.com/mars-probe-destroyed-...,mars probe destroyed by orbiting spielberg-gat...,1


In [42]:
dataset = dataset.drop_duplicates()

In [43]:
dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28497,https://www.theonion.com/tyson-holds-contest-t...,tyson holds contest to let fans submit new ide...,1
28509,https://politics.theonion.com/increasingly-coc...,increasingly cocky bernie sanders announces he...,1
28520,https://www.theonion.com/cash-strapped-zuckerb...,cash-strapped zuckerberg forced to sell 11 mil...,1
28544,https://local.theonion.com/grocery-store-bar-a...,grocery store bar actually has great little ha...,1


In [44]:
datatest = pd.read_csv('https://raw.githubusercontent.com/NQT2906/CS114.K21/Sarcasm/Sarcasm/Sarcasm_Combine.csv')

datatest = datatest.drop(datatest.columns[[0]], axis=1)  # df.columns is zero-based pd.Index 

datatest

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28612,https://www.theonion.com/tyson-holds-contest-t...,tyson holds contest to let fans submit new ide...,1
28613,https://politics.theonion.com/increasingly-coc...,increasingly cocky bernie sanders announces he...,1
28614,https://www.theonion.com/cash-strapped-zuckerb...,cash-strapped zuckerberg forced to sell 11 mil...,1
28615,https://local.theonion.com/grocery-store-bar-a...,grocery store bar actually has great little ha...,1


# **Split data to train and test (0.8 train and 0.2 test)**

In [45]:
X = dataset.iloc[:, 1].values
Y = dataset.iloc[:, 2].values

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state = 0)

# **Preprocessing**

In [13]:
corpus_train = []
for review in X_train: # Duyệt qua từng phần tử trong X_train.values
  review = re.sub('[^a-zA-Z]', ' ', review) #remove word not alphabet
  review = review.split() # split string to word
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')] # Loại bỏ các từ không có ý nghĩa
                                                                                        # Chuyển các từ về dạng nguyên mẫu
  review = ' '.join(review) # Ghép các từ sau khi xử lý lại thành 1 câu
  corpus_train.append(review)

In [14]:
corpus_test = []
for review in X_test:
  review = re.sub('[^a-zA-Z]', ' ', review) #remove word not alphabet
  review = review.split() # split string to word
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [15]:
print(np.shape(corpus_train))
print(np.shape(corpus_test))

(22893,)
(5724,)


# **Create dictionary for word in train and test**

In [47]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train).toarray()
trainVocab = cv.vocabulary_ #export tập từ vựng

In [130]:
cv = CountVectorizer(vocabulary=trainVocab)
X_test = cv.fit_transform(X_test).toarray()

In [18]:
print(X_test.shape)
print(Y_test.shape)

(5724, 23862)
(5724,)


# **Choose model**

In [ ]:
model = MultinomialNB()
model.fit(X_train, Y_train)

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

In [ ]:
print("Score of model (MultinomialNB) in train set:", model.score(X_train, Y_train))
print("Score of model (MultinomialNB) in test set:", model.score(X_test, Y_test))

print("Accuracy score of model (MultinomialNB) in train set:",accuracy_score(Y_train, Y_train_pred))
print("Accuracy score of model (MultinomialNB) in test set:", accuracy_score(Y_test, Y_test_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(random_state = 0)
model_log.fit(X_train, Y_train)

Y_train_pred = model_log.predict(X_train)
Y_test_pred = model_log.predict(X_test)

In [ ]:
print("Score of model (LogisticRegression) in train set:", model_log.score(X_train, Y_train))
print("Score of model (LogisticRegression) in test set:", model_log.score(X_test, Y_test))


print("Accuracy score of model (LogisticRegression) in train set:", accuracy_score(Y_train, Y_train_pred))
print("Accuracy score of model (LogisticRegression) in test set:", accuracy_score(Y_test, Y_test_pred))

# **Save model**

In [47]:
import pickle

In [48]:
filename = 'MultinomialNB_Sarcasm.sav'
pickle.dump(model, open(filename, 'wb'))

In [49]:
filename = 'Logistic_Regression_Sarcasm.sav'
pickle.dump(model_log, open(filename, 'wb'))

# **Load model and test new data**

**Import library**

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

**Load model**

In [36]:
import pickle

model_NB = pickle.load(open('/content/Logistic_Regression_Sarcasm.sav', 'rb'))

FileNotFoundError: ignored

In [30]:
datareal = pd.read_csv('Headline_Case_study.csv')

datareal.to_csv('Headline_Case_study_1.csv')

In [31]:
datareal = pd.read_csv('Headline_Case_study.csv')

X_real = datareal.iloc[:, 0].values
Y_real = datareal.iloc[:, 1].values

dictionary = pd.read_csv('Dictionary.csv')

#dictionary = dictionary.to_dict('split')

In [32]:
dictionary = dict(dictionary.values.tolist())

In [33]:
cv = CountVectorizer(vocabulary=dictionary)
X_real = cv.fit_transform(X_real).toarray()

In [34]:
Y_real_pred = model_NB.predict(X_real)

In [35]:
print("Score of model (MultinomialNB) in test set:", model_NB.score(X_real, Y_real))

Score of model (MultinomialNB) in test set: 0.7496867952894012
